<a href="https://colab.research.google.com/github/aluu999/Fizyka-Kwantowa-Projekt/blob/main/tunelowanie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy as sp
from IPython.display import HTML
import matplotlib.pyplot as plt
from matplotlib import animation
from scipy.sparse import linalg as ln

n_points = 1000
x, dx = sp.linspace(-100, 100, n_points, retstep=True) 
k = 2.0                                     
sigma = 2.0            # szerokość dzwona
x0 = -50                   # tu start
x_start = 0.0
x_end = 3.0

psi0 =(1/(2*np.pi*sigma**2))**(1/4) *np.exp(-((x-x0)/(2.0 * sigma))**2) * np.exp(1j * k * x)


# Potencjał 
V_height = 2.0

V = np.zeros(x.shape)
for i, _x in enumerate(x):
    if x_start <= _x <= x_end :
        V[i] = V_height
    else: 
        V[i] = 0.0


h_diag = sp.ones(n_points) / dx**2 + V
h_non_diag = sp.ones(n_points - 1) * (-0.5 / dx**2)
hamiltonian = sp.sparse.diags([h_diag, h_non_diag, h_non_diag], [0, 1, -1])
    

dt = 0.5
c1 = (sp.sparse.eye(x.size) - dt / 2.0j * hamiltonian).tocsc()
c2 = (sp.sparse.eye(x.size) + dt / 2.0j * hamiltonian).tocsc() 
cn = ln.inv(c1).dot(c2).tocsr()

def psit():
    global psi0, cn
    psi0 = cn.dot(psi0)
    prob = np.abs(psi0)**2
    return prob


# Animacja
fig = plt.figure()
ax = plt.axes(xlim = (-100,100), ylim = (0.0, 0.5))
line1, = ax.plot([], [])

plt.hlines(0.2, 0, 3, color = 'red')
plt.hlines(0, 0, 3, color = 'red')
plt.vlines(0, 0, 0.2, color = 'red')
plt.vlines(3, 0, 0.2, color = 'red')

def animate(i):
    line1.set_data(x, psit())
    return line1,


anim = animation.FuncAnimation(fig, animate, frames=150, interval=50, blit=True)
anim.save('gotowe.mp4', fps=15, extra_args=['-vcodec', 'libx264'], dpi=300)
